## Reduction: the sum of the elements of an array

In [ ]:
import sys
import numpy as np

# Valor por defecto en caso de ejecución manual
value = 5 * 10**7 

# Comprobamos si se ha pasado un argumento por línea de comandos
if len(sys.argv) > 1:
    try:
        # Usamos float y luego int para permitir notación científica como 1e8
        value = int(float(sys.argv[1]))
        print(f"Valor recibido por línea de comandos: {value}")
    except ValueError:
        print(f"Error en el parámetro. Usando valor por defecto: {value}")

X = np.random.rand(value)

In [1]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Para imprimir los primeros valores del array

#print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)

print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")

tiempo_secuencial_real = tiempo.average


Time taken by reduction operation using a function: 4.78 s ± 16.9 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 25001639.82326035

Time taken by reduction operation using numpy.sum(): 18.6 ms ± 27.4 μs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.sum(): 25001639.82326865 
 


In [2]:
#a)paquete multiprocessing

from multiprocessing import Pool
import numpy as np
import time

# Definimos los niveles de paralelismo a probar
procesos_a_probar = [2, 4]

for n_procs in procesos_a_probar:
    print(f"--- Evaluando con {n_procs} procesos ---")
    
    # a) Dividir el array en tantos sub-arrays como procesos
    chunks = np.array_split(X, n_procs)
    
    # b) Crear el pool de procesos
    # c) Usar Map para enviar cada sub-array a la función reduc_operation
    start_time = time.time()
    with Pool(processes=n_procs) as pool:
        partial_results = pool.map(reduc_operation, chunks)
    
    # d) Reducir (sumar) los resultados parciales devueltos
    total_sum_mp = sum(partial_results)
    end_time = time.time()
    
    tiempo_mp = end_time - start_time
    print(f"Tiempo con {n_procs} procesos: {tiempo_mp:.4f} s")
    # Ahora usamos la variable que definimos en la celda anterior
    print(f"Mejora (Speedup): {tiempo_secuencial_real / tiempo_mp:.2f}x\n")

--- Evaluando con 2 procesos ---
Tiempo con 2 procesos: 3.2943 s
Resultado de la suma: 25001639.823269553
Mejora (Speedup): 1.44x

--- Evaluando con 4 procesos ---
Tiempo con 4 procesos: 1.8038 s
Resultado de la suma: 25001639.82327029
Mejora (Speedup): 2.63x



In [3]:
#b) paquete numba

from numba import njit, prange

# 1. Versión compilada estándar con @njit
@njit
def reduc_operation_njit(A):
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# 2. Versión compilada con paralelismo automático (@njit(parallel=True))
@njit(parallel=True)
def reduc_operation_numba_par(A):
    s = 0
    # Usamos prange para indicar que el bucle se puede paralelizar
    for i in prange(A.size):
        s += A[i]
    return s

# Medición Numba njit
print("--- Tiempo con Numba @njit (Secuencial compilado) ---")
# La primera ejecución incluye el tiempo de compilación, %timeit lo ignora
%timeit -r 2 -o reduc_operation_njit(X)
print(f"Resultado: {reduc_operation_njit(X)}\n")

# Medición Numba parallel
print("--- Tiempo con Numba @njit(parallel=True) ---")
%timeit -r 2 -o reduc_operation_numba_par(X)
print(f"Resultado: {reduc_operation_numba_par(X)}\n")

--- Tiempo con Numba @njit (Secuencial compilado) ---
49 ms ± 91.7 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 25001639.82326035

--- Tiempo con Numba @njit(parallel=True) ---
11.4 ms ± 6.76 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 25001639.82326856



RESULTADOS

Convirtiendo notebook de alumno01...
/nas/hdd-0/modules/anaconda3-2025/bin/jupyter-nbconvert:7: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  from nbconvert.nbconvertapp import main
[NbConvertApp] Converting notebook reduc-operation-array-par-alumno01.ipynb to script
[NbConvertApp] Writing 3402 bytes to reduc-operation-array-par-alumno01.py
 
==============================================================
    INICIANDO BATERÍA PARA TAMAÑO: 100000000
==============================================================
 
Configuración: 1 Núcleos
Valor recibido por línea de comandos: 100000000
Time taken by reduction operation using a function: 18.1 s ± 129 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 49998306.22171873

Time taken by reduction operation using numpy.sum(): 64.1 ms ± 3.24 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.sum(): 49998306.22172294 
 
--- Evaluando con 2 procesos ---
Tiempo con 2 procesos: 12.1006 s
Mejora (Speedup): 0.01x

--- Evaluando con 4 procesos ---
Tiempo con 4 procesos: 6.6372 s
Mejora (Speedup): 0.01x

--- Tiempo con Numba @njit (Secuencial compilado) ---
115 ms ± 9.38 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 49998306.22171873

--- Tiempo con Numba @njit(parallel=True) ---
23.8 ms ± 665 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 49998306.22172344

 
Configuración: 2 Núcleos
Valor recibido por línea de comandos: 100000000
Time taken by reduction operation using a function: 17.1 s ± 54.9 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 50001519.78325166

Time taken by reduction operation using numpy.sum(): 64.1 ms ± 48.7 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.sum(): 50001519.78325853 
 
--- Evaluando con 2 procesos ---
Tiempo con 2 procesos: 11.0521 s
Mejora (Speedup): 0.01x

--- Evaluando con 4 procesos ---
Tiempo con 4 procesos: 6.1077 s
Mejora (Speedup): 0.01x

--- Tiempo con Numba @njit (Secuencial compilado) ---
115 ms ± 2.91 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 50001519.78325166

--- Tiempo con Numba @njit(parallel=True) ---
23.8 ms ± 135 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 50001519.78325865

 
Configuración: 4 Núcleos
Valor recibido por línea de comandos: 100000000
Time taken by reduction operation using a function: 17.4 s ± 148 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 49997971.75899098

Time taken by reduction operation using numpy.sum(): 64.1 ms ± 4.85 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.sum(): 49997971.75898186 
 
--- Evaluando con 2 procesos ---
Tiempo con 2 procesos: 11.0716 s
Mejora (Speedup): 0.01x

--- Evaluando con 4 procesos ---
Tiempo con 4 procesos: 6.1351 s
Mejora (Speedup): 0.01x

--- Tiempo con Numba @njit (Secuencial compilado) ---
115 ms ± 4.3 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 49997971.75899098

--- Tiempo con Numba @njit(parallel=True) ---
27.3 ms ± 3.01 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 49997971.75898212

 
Configuración: 8 Núcleos
Valor recibido por línea de comandos: 100000000
Time taken by reduction operation using a function: 17.3 s ± 20.5 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 50001609.02672845

Time taken by reduction operation using numpy.sum(): 64.1 ms ± 34.8 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.sum(): 50001609.02673872 
 
--- Evaluando con 2 procesos ---
Tiempo con 2 procesos: 11.2137 s
Mejora (Speedup): 0.01x

--- Evaluando con 4 procesos ---
Tiempo con 4 procesos: 6.1685 s
Mejora (Speedup): 0.01x

--- Tiempo con Numba @njit (Secuencial compilado) ---
115 ms ± 100 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 50001609.02672845

--- Tiempo con Numba @njit(parallel=True) ---
25.2 ms ± 922 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 50001609.026738696

 
==============================================================
    INICIANDO BATERÍA PARA TAMAÑO: 1000000000
==============================================================
 
Configuración: 1 Núcleos
Valor recibido por línea de comandos: 1000000000
Time taken by reduction operation using a function: 2min 52s ± 557 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 500011320.927258

Time taken by reduction operation using numpy.sum(): 640 ms ± 206 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Now, the result using numpy.sum(): 500011320.9264929 
 
--- Evaluando con 2 procesos ---
Tiempo con 2 procesos: 110.8169 s
Mejora (Speedup): 0.01x

--- Evaluando con 4 procesos ---
Tiempo con 4 procesos: 61.9578 s
Mejora (Speedup): 0.01x

--- Tiempo con Numba @njit (Secuencial compilado) ---
1.15 s ± 1.11 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 500011320.927258

--- Tiempo con Numba @njit(parallel=True) ---
341 ms ± 9.64 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 500011320.9264919

 
Configuración: 2 Núcleos
Valor recibido por línea de comandos: 1000000000
Time taken by reduction operation using a function: 2min 52s ± 499 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 499998362.6681816

Time taken by reduction operation using numpy.sum(): 641 ms ± 263 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Now, the result using numpy.sum(): 499998362.6679737 
 
--- Evaluando con 2 procesos ---
Tiempo con 2 procesos: 120.0972 s
Mejora (Speedup): 0.01x

--- Evaluando con 4 procesos ---
Tiempo con 4 procesos: 65.7615 s
Mejora (Speedup): 0.01x

--- Tiempo con Numba @njit (Secuencial compilado) ---
1.15 s ± 1.53 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 499998362.6681816

--- Tiempo con Numba @njit(parallel=True) ---
339 ms ± 8.36 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 499998362.6679723

 
Configuración: 4 Núcleos
Valor recibido por línea de comandos: 1000000000
Time taken by reduction operation using a function: 2min 53s ± 733 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 500011324.0201602

Time taken by reduction operation using numpy.sum(): 640 ms ± 70.9 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Now, the result using numpy.sum(): 500011324.02028614 
 
--- Evaluando con 2 procesos ---
Tiempo con 2 procesos: 112.5218 s
Mejora (Speedup): 0.01x

--- Evaluando con 4 procesos ---
Tiempo con 4 procesos: 64.4065 s
Mejora (Speedup): 0.01x

--- Tiempo con Numba @njit (Secuencial compilado) ---
1.15 s ± 1.05 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 500011324.0201602

--- Tiempo con Numba @njit(parallel=True) ---
339 ms ± 10 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 500011324.02028835

 
Configuración: 8 Núcleos
Valor recibido por línea de comandos: 1000000000
Time taken by reduction operation using a function: 2min 52s ± 503 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 499999117.1594029

Time taken by reduction operation using numpy.sum(): 641 ms ± 239 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Now, the result using numpy.sum(): 499999117.1589212 
 
--- Evaluando con 2 procesos ---
Tiempo con 2 procesos: 110.9729 s
Mejora (Speedup): 0.01x

--- Evaluando con 4 procesos ---
Tiempo con 4 procesos: 61.7718 s
Mejora (Speedup): 0.01x

--- Tiempo con Numba @njit (Secuencial compilado) ---
1.15 s ± 1.2 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 499999117.1594029

--- Tiempo con Numba @njit(parallel=True) ---
332 ms ± 14.3 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 499999117.1589212

Trabajo finalizado con éxito.


d) Análisis de resultados obtenidos en Mendel

Tras ejecutar la batería de pruebas para tamaños de array de 108 (100 millones) y 109 (1.000 millones) de elementos, se presentan los resultados de rendimiento promedio:
Tabla Resumen de Tiempos (Tamaño 109 - 1 Billón de elementos)
Método	1 Núcleo	2 Núcleos	4 Núcleos	8 Núcleos
Original (Python)	172.0 s	172.0 s	173.0 s	172.0 s
Multiprocessing (Pool)	---	110.8 s	61.9 s	---
Numba (@njit)	1.15 s	1.15 s	1.15 s	1.15 s
Numba (Parallel)	0.34 s	0.34 s	0.34 s	0.33 s

1. Código Original (Python Puro)

    Comportamiento: Es el método más lento, tardando casi 3 minutos para procesar 109 elementos.

    Escalabilidad: Nula. El tiempo se mantiene constante (~172s) sin importar cuántos núcleos asigne SLURM. Esto se debe a que el bucle for es secuencial y el intérprete de Python (GIL) impide usar más de un núcleo.

2. Multiprocessing (Pool)

    Comportamiento: Se observa una mejora significativa respecto al original. Al pasar de 2 a 4 procesos, el tiempo se reduce de 110s a 61s (una aceleración casi lineal de 1.8x).

    Nota sobre el Speedup: En los logs aparece "0.01x" debido a que el código comparaba el tiempo contra un valor fijo de la práctica anterior. Calculando el speedup real sobre el tiempo secuencial de 172s, la mejora con 4 núcleos es de 2.7x.

3. Numba (@njit y Parallel)

    Compilación JIT: Solo con @njit (secuencial), el tiempo baja de 172 segundos a 1.15 segundos (una mejora de 150 veces). Esto demuestra la potencia de convertir Python en lenguaje máquina antes de ejecutar.

    Paralelismo de hilos: Con parallel=True, el tiempo cae a 0.33 segundos. Es el método más eficiente de todos, superando a multiprocessing porque usa hilos ligeros en lugar de procesos pesados, evitando el coste de copiar datos en memoria.

4. Precisión Numérica

    Se observa que los resultados de la suma varían ligeramente a partir del 7º decimal (ej: ...927258 vs ...926491). Como se analizó anteriormente, esto es un efecto esperado de la aritmética de punto flotante al cambiar el orden de las sumas en los hilos paralelos.
